In [3]:
import os 


In [4]:
!pwd

/Users/sanjaymahto/Chicken-Disease-Classifiaction/research


In [5]:
os.chdir('../')

In [6]:
!pwd

/Users/sanjaymahto/Chicken-Disease-Classifiaction


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights: str
    params_classes: int

In [8]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
import config


In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
        root_dir = Path(config.root_dir),
        base_model_path = Path(config.base_model_path),
        updated_base_model_path = Path(config.updated_base_model_path),
        params_image_size = self.params.IMAGE_SIZE,
        params_learning_rate = self.params.LEARNING_RATE,
        params_include_top = self.params.INCLUDE_TOP,
        params_weights = self.params.WEIGHTS,
        params_classes = self.params.ClASSES
        )
        return prepare_base_model_config
        

In [10]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 

In [11]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size, 
            weights=self.config.params_weights, 
            include_top=self.config.params_include_top,
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model) # I am saving the model in the artifacts folder
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                layer.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes, 
            activation='softmax'
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs = prediction
        )
        # model = tf.keras.Sequential([
        #     model,
        #     tf.keras.layers.Flatten(),
        #     tf.keras.layers.Dense(classes, activation='softmax')
        # ])
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    print(e)
    print(f"Error in preparing the base model{e}")

[2024-08-06 12:56:36,699: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/config/config.yaml loaded successfully]
[2024-08-06 12:56:36,701: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/params.yaml loaded successfully]
[2024-08-06 12:56:36,701: INFO: common: created directory at: artifacts]
[2024-08-06 12:56:36,702: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-08-06 12:56:37,064: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-08-06 12:56:37,190: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
[2024-08-06 12:56:37,192: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling bac

/Users/sanjaymahto/Chicken-Disease-Classifiaction/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
